In [1]:
import pandas as pd
advertisers = pd.read_json('data/advertisers_dec1.json', lines=True)

In [2]:
adv = pd.concat(
    [pd.json_normalize(advertisers["userDetailVo"], sep="_", max_level=None),
    pd.json_normalize(advertisers["feedback"], sep="_", max_level=None)],
    axis=1)
adv["timestamp"] = advertisers["timestamp"]
adv.sort_values(by=["timestamp"], ascending=False, inplace=True)
adv.drop_duplicates(subset='userNo', keep='first', inplace=True); # unique Users

In [3]:
adv['netRate'] = adv['positiveRate'] - adv['negativeRate']

In [4]:
pos_bins = [-0.01, 0.01, 0.1, 0.5, 0.75, 0.80, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00]
bin_labels = [0, 0.01, 0.1, 0.5, 0.75, 0.80, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]

In [5]:
adv['pos_bin'] = pd.cut(adv['positiveRate'], pos_bins)
adv['neg_bin'] = pd.cut(adv['negativeRate'], pos_bins)
adv['net_bin'] = pd.cut(adv['netRate'], pos_bins)
adv['pos_bin_label'] = pd.cut(adv['positiveRate'], pos_bins, labels=bin_labels)
adv['neg_bin_label'] = pd.cut(adv['negativeRate'], pos_bins, labels=bin_labels)
adv['net_bin_label'] = pd.cut(adv['netRate'], pos_bins, labels=bin_labels)

In [13]:
pos = adv[["pos_bin_label", "pos_bin", "userStatsRet_finishRateLatest30day", "userStatsRet_avgPayTimeOfLatest30day", 
"userStatsRet_avgReleaseTimeOfLatest30day", "userStatsRet_completedOrderNum", "userStatsRet_completedBuyOrderNum",
"userStatsRet_completedSellOrderNum", "userStatsRet_counterpartyCount"]].groupby(["pos_bin"]).mean()
neg = adv[["neg_bin_label", "neg_bin", "userStatsRet_finishRateLatest30day", "userStatsRet_avgPayTimeOfLatest30day", 
"userStatsRet_avgReleaseTimeOfLatest30day", "userStatsRet_completedOrderNum", "userStatsRet_completedBuyOrderNum",
"userStatsRet_completedSellOrderNum", "userStatsRet_counterpartyCount"]].groupby(["neg_bin"]).mean()
net = adv[["net_bin_label", "net_bin", "userStatsRet_finishRateLatest30day", "userStatsRet_avgPayTimeOfLatest30day", 
"userStatsRet_avgReleaseTimeOfLatest30day", "userStatsRet_completedOrderNum", "userStatsRet_completedBuyOrderNum",
"userStatsRet_completedSellOrderNum", "userStatsRet_counterpartyCount"]].groupby(["net_bin"]).mean()

/var/folders/s7/cbvrkwmd5vj_3ymczzxpsc940000gn/T/ipykernel_18492/4271574207.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pos = adv[["pos_bin_label", "pos_bin", "userStatsRet_finishRateLatest30day", "userStatsRet_avgPayTimeOfLatest30day",
/var/folders/s7/cbvrkwmd5vj_3ymczzxpsc940000gn/T/ipykernel_18492/4271574207.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  neg = adv[["neg_bin_label", "neg_bin", "userStatsRet_finishRateLatest30day", "userStatsRet_avgPayTimeOfLatest30day",
/var/folders/s7/cbvrkwmd5vj_3ymczzxpsc940000gn/T/ipykernel_18492/4271574207.py:7: FutureWarning: The default value of numeri

In [14]:
pos.to_csv("csv/positive_ratings_avgs.csv")
neg.to_csv("csv/negative_ratings_avgs.csv")
net.to_csv("csv/net_ratings_avgs.csv")